In [ ]:
import pypsa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = [10, 5]
import seaborn as sns
import matplotlib.dates as mdates

### Optimized

In [ ]:
# Import solved network
nw = pypsa.Network(
    "/Users/nomio/Documents/REU/Fall_2324/test/pypsa-usa/workflow/results/western/networks/elec_s_30_ec_lv1.25_Co2L1.25.nc"
)
# columns_to_drop = [col for col in nw.generators_t["p"].columns if 'Arizona' in col]
# ba_carrier = nw.generators_t["p"].drop(columns_to_drop, axis=1)
optimized = (
    ba_carrier.groupby(axis="columns", by=nw.generators["carrier"])
    .sum()
    .loc["2019-01-02 00:00:00":"2019-12-30 23:00:00"]
)
optimized["CCGT"] = optimized["CCGT"] + optimized["OCGT"]
optimized_final = optimized.drop(["OCGT", "offwind", "load"], axis=1)
op_f = optimized_final.rename(
    columns={
        "CCGT": "Natural gas",
        "hydro": "Hydro",
        "oil": "Oil",
        "onwind": "Onshore wind",
        "solar": "Solar",
        "nuclear": "Nuclear",
        "coal": "Coal",
        "geothermal": "Other",
    }
)
optimized = op_f / 1e3
optimized

In [ ]:
nw = pypsa.Network(
    "/Users/nomio/Documents/REU/Fall_2324/pypsa-usa-n/workflow/results/western/networks/elec_s_30_ec_lv1.25_Co2L1.25.nc"
)
nw.generators_t["p"]

In [ ]:
nw = pypsa.Network(
    "/Users/nomio/Documents/REU/Fall_2324/test/pypsa-usa/workflow/results/western/networks/elec_s_100_ec_lv1.25_Co2L1.25.nc"
)
# columns_to_drop = [col for col in nw.generators_t["p"].columns if 'Arizona' in col]
# ba_carrier = nw.generators_t["p"].drop(columns_to_drop, axis=1)
optimized = (
    ba_carrier.groupby(axis="columns", by=nw.generators["carrier"])
    .sum()
    .loc["2019-01-02 00:00:00":"2019-12-30 23:00:00"]
)
optimized["CCGT"] = optimized["CCGT"] + optimized["OCGT"]
optimized_final = optimized.drop(["OCGT", "offwind", "load"], axis=1)
op_f = optimized_final.rename(
    columns={
        "CCGT": "Natural gas",
        "hydro": "Hydro",
        "oil": "Oil",
        "onwind": "Onshore wind",
        "solar": "Solar",
        "nuclear": "Nuclear",
        "coal": "Coal",
        "geothermal": "Other",
    }
)
optimized = op_f / 1e3
optimized

In [ ]:
nw = pypsa.Network(
    "/Users/nomio/Documents/REU/Fall_2324/pypsa-usa-n/workflow/results/western/networks/elec_s_30_ec_lv1.25_Co2L0.75.nc"
)
# columns_to_drop = [col for col in nw.generators_t["p"].columns if 'Arizona' in col]
# ba_carrier = nw.generators_t["p"].drop(columns_to_drop, axis=1)
nw.generators_t["p"]

In [ ]:
# nw = pypsa.Network("/Users/nomio/Documents/REU/Fall_2324/test/pypsa-usa/workflow/results/western/networks/elec_s_100_ec_lv1.25_Co2L1.25.nc")
from collections import OrderedDict

buses = []
for i in range(nw.generators.bus.size):
    if nw.generators.bus[i] not in buses:
        buses.append(nw.generators.bus[i])
buses_clean = [ba.split("0")[0] for ba in buses]
buses_clean = [ba.split("-")[0] for ba in buses_clean]
buses = list(OrderedDict.fromkeys(buses_clean))
buses
# buses.pop(1)

## Historic

In [ ]:
selected_cols = [
    "Balancing Authority",
    "UTC Time at End of Hour",
    "Net Generation (MW) from Natural Gas (Adjusted)",
    "Net Generation (MW) from Coal (Adjusted)",
    "Net Generation (MW) from Nuclear (Adjusted)",
    "Net Generation (MW) from All Petroleum Products (Adjusted)",
    "Net Generation (MW) from Hydropower and Pumped Storage (Adjusted)",
    "Net Generation (MW) from Solar (Adjusted)",
    "Net Generation (MW) from Wind (Adjusted)",
    "Net Generation (MW) from Other Fuel Sources (Adjusted)",
]
historic_first = pd.read_csv(
    "/Users/nomio/Documents/REU/Fall_2324/pypsa-usa-n/workflow/resources/eia/6moFiles/EIA930_BALANCE_2019_Jan_Jun.csv",
    index_col=[0, 1],
    header=0,
    parse_dates=True,
    date_format="%m/%d/%Y %I:%M:%S %p",
    usecols=selected_cols,
)
historic_first_df = (
    historic_first.loc[buses].fillna(0).replace({",": ""}, regex=True).astype(float)
)
historic_second = pd.read_csv(
    "/Users/nomio/Documents/REU/Fall_2324/pypsa-usa-n/workflow/resources/eia/6moFiles/EIA930_BALANCE_2019_Jul_Dec.csv",
    index_col=[0, 1],
    header=0,
    parse_dates=True,
    date_format="%m/%d/%Y %I:%M:%S %p",
    usecols=selected_cols,
)
historic_second_df = (
    historic_second.loc[buses].fillna(0).replace({",": ""}, regex=True).astype(float)
)
historic = (
    pd.concat([historic_first_df, historic_second_df], axis=0)
    .groupby(["UTC Time at End of Hour"])
    .sum()
)
historic[historic < 0] = 0
col_names = {
    "Net Generation (MW) from Natural Gas (Adjusted)": "Natural gas",
    "Net Generation (MW) from Hydropower and Pumped Storage (Adjusted)": "Hydro",
    "Net Generation (MW) from All Petroleum Products (Adjusted)": "Oil",
    "Net Generation (MW) from Wind (Adjusted)": "Onshore wind",
    "Net Generation (MW) from Solar (Adjusted)": "Solar",
    "Net Generation (MW) from Nuclear (Adjusted)": "Nuclear",
    "Net Generation (MW) from Coal (Adjusted)": "Coal",
    "Net Generation (MW) from Other Fuel Sources (Adjusted)": "Other",
}
historic = historic.rename(columns=col_names)
rearrange = [
    "Natural gas",
    "Coal",
    "Other",
    "Hydro",
    "Nuclear",
    "Oil",
    "Onshore wind",
    "Solar",
]
historic = historic[rearrange].loc["2019-01-02 00:00:00":"2019-12-30 23:00:00"] / 1e3

In [ ]:
historic - optimized
historic[historic.index.duplicated(keep=False)]
historic.loc["2019-07-01 00:00:00":"2019-07-01 10:00:00"]

## Plot

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(9, 9))
colors = [
    "purple",
    "dimgray",
    "brown",
    "royalblue",
    "chocolate",
    "green",
    "lightskyblue",
    "crimson",
]
kwargs = dict(color=colors, legend=False, ylabel="Production [GW]", xlabel="")
optimized.resample("1D").sum().plot.area(ax=axes[0], **kwargs, title="Optimized")
historic.resample("1D").sum().plot.area(ax=axes[1], **kwargs, title="Historic")

diff = (optimized - historic).fillna(0).resample("1D").sum()
diff.clip(lower=0).plot.area(
    ax=axes[2], **kwargs, title="$\Delta$ (Optimized - Historic)"
)
lim = axes[2].get_ylim()[1]
diff.clip(upper=0).plot.area(ax=axes[2], **kwargs)
axes[2].set_ylim(bottom=-lim, top=lim)

h, l = axes[0].get_legend_handles_labels()
fig.legend(
    h[::-1],
    l[::-1],
    loc="center left",
    bbox_to_anchor=(1, 0.5),
    ncol=1,
    frameon=False,
    labelspacing=1,
)

In [ ]:
data = pd.concat([historic, optimized], keys=["Historic", "Optimized"], axis=1)
data.columns.names = ["Kind", "Carrier"]
data

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

df = data.groupby(level=["Kind", "Carrier"], axis=1).sum().sum().unstack().T
df = df / 1e3  # TWh
df.plot.barh(ax=ax, xlabel="Electricity Production [TWh]", ylabel="")
ax.grid(axis="y")

In [ ]:
fig, ax = plt.subplots(figsize=(6, 10))

df = data.sum() / 1e3  # TWh
df = df["Optimized"] - df["Historic"]
df = df.dropna().sort_values()
# df = pd.concat([df.iloc[:5], df.iloc[-5:]])
# c = colors[df.index.get_level_values(0)]
df.plot.barh(xlabel="Optimized Production - Historic Production [TWh]", ax=ax)
ax.set_title("Strongest Deviations")
ax.grid(axis="y")

In [ ]:
df

## Ordering

In [ ]:
order = (historic.diff().abs().sum() / historic.sum()).sort_values().index
order

In [ ]:
optimized = optimized.reindex(order, axis=1, level=1)
historic = historic.reindex(order, axis=1, level=1)

In [ ]:
optimized

In [ ]:
fig, ax = plt.subplots(figsize=(6, 10))
diff = (optimized - historic).sum() / 1e3  # in TW
diff = diff.dropna().sort_values()
diff.plot.barh(xlabel="Optimized Production - Historic Production [TWh]", ax=ax)
ax.set_title("Strongest Deviations")
ax.grid(axis="y")

In [ ]:
optimized["Solar"].sum()